<a href="https://colab.research.google.com/github/defeng1/asl-assistant/blob/back-end/model_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
# https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/

import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import pathlib
from tensorflow.keras import layers
from google.colab import drive
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
#Parameters
img_height = 200
img_width = 200

num_classes = 29

In [4]:
# Function to create model, required for KerasClassifier
def create_model(learn_rate=0.01):
	# create model
	model = tf.keras.Sequential([
		layers.experimental.preprocessing.Rescaling(1./255),
		layers.Conv2D(32, 3, activation='relu'),
		layers.MaxPooling2D(),
		layers.Conv2D(32, 3, activation='relu'),
		layers.MaxPooling2D(),
		layers.Conv2D(32, 3, activation='relu'),
		layers.MaxPooling2D(),
		layers.Flatten(),
		layers.Dense(128, activation='relu'),
		layers.Dense(num_classes)
	])
 
	# Compile model
	adam = tf.keras.optimizers.Adam(learning_rate=learn_rate)
	model.compile(
		optimizer = adam,
		loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True),
		metrics=['sparse_categorical_accuracy'])
	
	return model

In [19]:
#Importing the dataset
data_dir = "/content/gdrive/My Drive/Capstone/Dataset/test_set"
data_dir = pathlib.Path(data_dir)

ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  image_size=(img_height, img_width))

Found 8702 files belonging to 29 classes.


In [35]:
ds_iter = ds.as_numpy_iterator()
X, Y = ds_iter.next()
count = 0
for x, y in ds.as_numpy_iterator():
  if count == 4:
    break
  inputs = np.add(X, x)
  inputs = np.add(Y, y)
  count += 1

print("input is {}".format(X.))
print("output is {}".format(type(Y)))
print(count)

input is <class 'numpy.ndarray'>
output is <class 'numpy.ndarray'>
4


In [17]:
# dataset = numpy.loadtxt("/content/gdrive/My Drive/Capstone/Dataset/model optimizations/pima-indians-diabetes.data.csv", delimiter=",")
# # split into input (X) and output (Y) variables
# X = dataset[:,0:8]
# Y = dataset[:,8]

# create model
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)

# define the grid search parameters
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [1, 5, 10, 15]
param_grid = dict(batch_size=batch_size, epochs=epochs, learn_rate=learn_rate)

# grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
# grid_result = grid.fit(X, Y)
grid_result = grid.fit(ds.as_numpy_iterator())

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

TypeError: ignored